In [3]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-dt9mzpb6
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-dt9mzpb6
     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 42.1 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 47.9 MB/s 
     |████████████████████████████████| 6.8 MB 30.5 MB/s 
     |████████████████████████████████| 596 kB 49.6 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8695 sha256=72360e98a26461c3251692b2144fe5f1b72b4c0e0878907498752f0aec51cd19
  Stored in directory: /tmp/pip-ephem-wheel-cache-03r5rb9v/wheels/ff/27/79/a53e92f32da44c8c4f51094ca156e6ff91103b7e48a341d62b
  Created wheel for python-Levenshtein: file

In [4]:
from urllib import request
import pandas as pd
import logging
import torch
import numpy as np
from collections import Counter
from ast import literal_eval

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import matplotlib.pyplot as plt

from parrot import Parrot
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [5]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [9]:
from dont_patronize_me import DontPatronizeMe
dpm = DontPatronizeMe('.', '.')
dpm.load_task1()
data = dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [10]:
def load_paragraph_ids():
    train_ids = pd.read_csv('train_semeval_parids-labels.csv')
    test_ids = pd.read_csv('dev_semeval_parids-labels.csv')
    train_ids.par_id = train_ids.par_id.astype(str)
    test_ids.par_id = test_ids.par_id.astype(str)
    print(f"training data size: {len(train_ids)}; test data size: {len(test_ids)}")
    return train_ids, test_ids
    
def rebuild_data_set(dpm, ids):
    rows = [] # will contain par_id, label and text
    for idx in range(len(ids)):  
        parid = ids.par_id[idx]

        # select row from original dataset
        text = dpm.loc[dpm.par_id == parid].text.values[0]
        label = dpm.loc[dpm.par_id == parid].label.values[0]
        orig_label = dpm.loc[dpm.par_id == parid].orig_label.values[0]
        art_id = dpm.loc[dpm.par_id == parid].art_id.values[0]
        keyword = dpm.loc[dpm.par_id == parid].keyword.values[0]
        country = dpm.loc[dpm.par_id == parid].country.values[0]
        rows.append({
            'par_id':parid,
            'text':text,
            'label':label,
            'orig_label': orig_label,
            'art_id': art_id,
            'keyword': keyword,
            'country': country
        })
    return pd.DataFrame(rows)

train_ids, _ = load_paragraph_ids()

train_data = rebuild_data_set(data, train_ids)
train_data

training data size: 8375; test data size: 2094


,par_id,text,label,orig_label,art_id,keyword,country
0,4341,"The scheme saw an estimated 150,000 children f...",1,4,@@17139403,poor-families,gb
1,4136,Durban 's homeless communities reconciliation ...,1,2,@@22273328,homeless,za
2,10352,The next immediate problem that cropped up was...,1,4,@@21102155,poor-families,lk
3,8279,Far more important than the implications for t...,1,2,@@21220476,vulnerable,nz
4,1164,To strengthen child-sensitive social protectio...,1,4,@@14727121,poor-families,gh
...,...,...,...,...,...,...,...
8370,8380,Rescue teams search for survivors on the rubbl...,0,0,@@3172947,refugee,gb
8371,8381,The launch of ' Happy Birthday ' took place la...,0,0,@@23593795,hopeless,ke
8372,8382,"The unrest has left at least 20,000 people dea...",0,0,@@9222273,homeless,pk
8373,8383,You have to see it from my perspective . I may...,0,0,@@25979256,hopeless,pk


In [11]:
pos_data = train_data[train_data.label==1]
pos_data = pos_data.reset_index()
pos_data

,index,par_id,text,label,orig_label,art_id,keyword,country
0,0,4341,"The scheme saw an estimated 150,000 children f...",1,4,@@17139403,poor-families,gb
1,1,4136,Durban 's homeless communities reconciliation ...,1,2,@@22273328,homeless,za
2,2,10352,The next immediate problem that cropped up was...,1,4,@@21102155,poor-families,lk
3,3,8279,Far more important than the implications for t...,1,2,@@21220476,vulnerable,nz
4,4,1164,To strengthen child-sensitive social protectio...,1,4,@@14727121,poor-families,gh
...,...,...,...,...,...,...,...,...
789,789,873,Citing the fact that these kids who died at Go...,1,2,@@20374243,poor-families,sg
790,790,10070,Fern ? ndez was a well-known philanthropist wh...,1,2,@@15573661,disabled,ng
791,791,6484,"Touched much by their plight , Commanding Offi...",1,2,@@2559173,homeless,lk
792,792,6249,She reiterated her ministry 's commitment to p...,1,2,@@1947926,women,gh


In [12]:
''' 
uncomment to get reproducable paraphrase generations
'''
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
torch.cuda.is_available()

True

In [14]:
with open("augment_positive.tsv", 'w') as f:
  for index, row in tqdm(pos_data.iterrows(), total=len(pos_data)):
    para_phrases = parrot.augment(input_phrase=row.text, 
                                  use_gpu=True, 
                                  diversity_ranker="levenshtein", 
                                  do_diverse=True, 
                                  max_return_phrases=10, 
                                  max_length=90, 
                                  adequacy_threshold=0.90, 
                                  fluency_threshold=0.90)
    if para_phrases is not None:
      for paraphrase, _ in para_phrases:
        line = f"{row.par_id}\t{row.art_id}\t{row.keyword}\t{row.country}\t{paraphrase}\t{row.orig_label}\n"
        f.write(line)
    else:
      print(para_phrases)

  4%|▍         | 30/794 [02:47<1:11:15,  5.60s/it]


TypeError: ignored

In [ ]:
with open("augment_positive.tsv", 'r') as f:
  print(len(f.readlines()))